In [45]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import os

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [46]:
base_dir = "/content/drive/MyDrive/WR"
data_dir = base_dir + "/WR Finder.v1i.yolov8"
train_dir = data_dir + "/train"
val_dir = data_dir + "/val"
test_dir = data_dir + "/test"
modes = {"train": train_dir, "val": val_dir, "test": test_dir}
def collect_data(mode):
  imgs = []
  labels = []
  path = modes[mode]
  img_path = os.path.join(path, "images")
  label_path = os.path.join(path, "labels")
  for fn in os.listdir(img_path):
    label_fn = fn[:-3] + "txt"
    img = cv2.imread(os.path.join(img_path, fn))
    with open(os.path.join(label_path, label_fn), "r") as f:
      label = f.readlines()
    imgs.append(img)
    labels.append([box.strip() for box in label])
  return imgs, labels

In [47]:
# filter out NFL logos and end zone coloring, trying to turn the image as green as possible
def filter_football_field(image):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_green = np.array([40, 40, 40])
    upper_green = np.array([70, 255, 255])

    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    # Find contours of green areas
    contours, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask for the football field by filling the contour areas
    field_mask = np.zeros_like(mask_green)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 5000:
            cv2.drawContours(field_mask, [contour], -1, 255, -1)

    # Invert the field mask
    field_mask = cv2.bitwise_not(field_mask)

    # Create a green background
    green_background = np.zeros_like(image, np.uint8)
    green_background[:] = (0, 255, 0)  # Green color

    # Apply the mask to the original image, and add green background to the filtered image
    filtered_image = cv2.bitwise_and(image, image, mask=field_mask)
    result = cv2.bitwise_or(filtered_image, green_background, mask=field_mask)

    return result

In [51]:
# Load the image
imgs, labels = collect_data("train")
image = imgs[0].copy()
cv2_imshow(image)

# Apply the filter
# filtered_image = filter_football_field(image)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/WR/WR Finder.v1i.yolov8/train/images'